# Goal and Purpose

Linear regression is the simplest and arguably most well-known form of machine learning.

Linear regression models, as in statistics, are concerned with minimizing error and making the most accurate predictions possible. A neural network can accomplish this by iteratively updating its internal parameters (weights) via a gradient descent algorithm. The Youtube educator 3B1B has a [great visualization](https://www.youtube.com/watch?v=IHZwWFHWa-w) of gradient descent in the context of machine learning models.

In this notebook, I will develop a simple single-layer model to "learn" how to predict the outcome of a linear function of the form y=mx+b. This excersize may seem trivial or unecessary ("I can produce a list of solutions for a linear function and plot it with a few lines of code, why do I need a neural network to try and learn how to do the same thing) but this will act as a backbone to build more complex neural networks for much **much** more complicated functions.

In [1]:
import numpy as np

## Simplest neural net
A neural network can be thought of as a function: input, transformation, and output. Therefore, in its most simple representation, a neural network can take a single input, produce an output, and by comparing that output with the known result, can update its internals to better approach the correct output value.

![simplest_NN.png](./fig/simplest_NN.png)

Here, x is some input number, the input is transformed via our neural network function which has parameters W and b (weight and bias). These parameters are subject to change based on how erroneous the network's output $\hat{y}$ is compared to the actual value we'd expect from input x.

The simple neural network has the following steps:
1. Initialize training input `x_train` and output `y_train`. The output here is the expected correct answer.
2. Initialize network parameters `W` and `b` . Here the weight array must correspond to the number of inputs. Since we only feed in one input at a time for now, the weights and bias arrays will have shape (1,1). The weight is initialized to a small random number.
3. Define our `cost` function. The "cost" can be thought of as the error between the expected output and our network's output. "Cost" and "Loss" are similar, though I believe the Loss function is the averaged error when considering a multitude of simultaneous inputs. We'll showcase this later, for now, each error calculation is refered to as the cost.
4. Calculate the components of the gradient of the `cost` function. In this case: $\frac{\delta W}{\delta C}$ and $\frac{\delta b}{\delta C}$.
5. Update the network parameters by reducing by a scaled amount of the gradient components. This is *gradient descent*. 
6. Repeat this process any number of times, called *epochs*. Return the parameters `W` and `b`. 
7. Use the model's updated parameters on *test data* to determine how accurate the trained model is.

In [239]:
# This is the function we want to have our neural network learn to model:
# y = 5x + 3
def F(x):
    f = 5.*x + 3.
    return f

# 1. initialize x_train, y_train
x_train=np.array([2.]).reshape(1,1)
y_train=F(x_train)

# 2. initialize network parameters
W = np.random.randn(1,1)*0.01
b = np.zeros(1).reshape(1,1)

# Setup an arbitrary function of the form that we are interested in
# This is also called the activation function of the neuron.
def P(W, b, x):
    f = np.dot(W.T,x) + b
    return f

# 3. Define cost function
def C(W,b,x,y):
    cost = (y-P(W,b,x))**2
    return cost

# 4,5,6. Gradient descent, update parameters, repeat
def model(W,b,x,y,epochs=5,learning_rate=0.05):
    for e in range(epochs):
        error=C(W,b,x,y)
        dw = 2.*(y-P(W,b,x))*(-1.0*x)
        db = -2.*(y-P(W,b,x))
        W -= dw*learning_rate
        b -= db*learning_rate
    return W,b
W,b = model(W,b,x_train,y_train,epochs=5,learning_rate=0.05)

print("y=",np.mean(W),"x +",np.mean(b))

# 7. Determine model's accuracy
x_test = np.array([5.])
y_test = F(x_test)

result = W*x_test + b
accuracy = 100-np.mean(np.abs(result - y_test))*100
print(result, y_test)
print(accuracy)


In [241]:
#x_train=np.arange(0,5)
#y_train=F(x_train)

# for e in range(5):
#     for i,x in enumerate(x_train):
#         cost = C(W,b,x,y_train[i])
#         dw = 2.*(y_train[i]-P(W,b,x))*(-1.0*x)
#         db = -2.*(y_train[i]-P(W,b,x))
#         W-=dw*0.05
#         b-=db*0.05
#         print(cost)
#         print(W,b)

y= 5.195246985532606 x + 2.596813878988607


[[27.72494767]] [28.]
72.49476744582033
